In [1]:
from scipy.optimize import curve_fit
import numpy as np
import pandas as pd
from datetime import datetime

from lmfit import Minimizer, Parameters, report_fit
import chart_studio.plotly as py

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import cufflinks as cf

In [2]:
def Cauchy_cumulative_hazard_fit(x,loc,scale,decaybase):
    decayterm=np.power(decaybase,(x-loc))
    decayterm[np.where((x-loc)<0)]=1.0
    z=(x-loc)/(scale*decayterm)
    CHF=-np.log(0.5 - np.arctan(z)/np.pi)
    return CHF

# define objective function: returns the array to be minimized
def Cauchy_cumulative_hazard_residual(params, x, data):
    """Model a decaying sine wave and subtract data."""
    amp = params['amp']
    loc = params['loc']
    scale = params['scale']
    decaybase=params['decaybase']
    decayterm=np.power(decaybase,(x-loc))
    decayterm[np.where((x-loc)<0)]=1.0
    z=(x-loc)/(scale*decayterm)
    CHF=-np.log(0.5 - np.arctan(z)/np.pi)
    model = amp * CHF
    return model - data

In [3]:
# https://data.humdata.org/dataset/coronavirus-covid-19-cases-data-for-china-and-the-rest-of-the-world
# worldTot.to_csv('worldTot.csv')
# datafile="data/COVID19/WHO COVID Cases Data - covid-19 historical cases by country.csv"
data00=pd.read_csv('worldTot.csv')

In [4]:
data00=data00.rename(columns={"Unnamed: 0":"date"})
dataWorld=data00.set_index('date')
dataWorld.head()

,cases
date,
1/22/20,555
1/23/20,654
1/24/20,941
1/25/20,1434
1/26/20,2118


In [5]:
start_date=dataWorld.index[0]
end_date=dataWorld.index[-1]
dateData=pd.date_range(start=start_date,end=end_date)
dataWorld['Date']=dateData
dataWorld=dataWorld.set_index('Date')


data=dataWorld['cases']
forecastDays=60

dateForecast= pd.date_range(start=end_date,periods=forecastDays+1)[1:] 
dateObsForecast=dateData.append(dateForecast)
#dateObsForecast

                         
# define objective function: returns the array to be minimized
  # normalize case data
last=data[-1]
data=data/last
  # set x values interval = 1day
dataLen=data.count()
x = np.linspace(1, dataLen, dataLen)

In [6]:
# data00=data00[['DateOfDataEntry','cum_conf','NewCase']]
# data00['DateOfDataEntry']=pd.to_datetime(data00['DateOfDataEntry'])
# dataWorld=data00.groupby('DateOfDataEntry').sum()

# start_date=dataWorld.index[0].date()
# end_date=dataWorld.index[-1].date()



# create a set of Parameters
params = Parameters()
params.add('amp', value=1)
params.add('scale', value=1, min=0.1)
params.add('loc', value=100, min=0)
params.add('decaybase',value=1,min=0.8,max=1.05)  # shparly increase value>1

# do fit, here with the default leastsq algorithm
minner = Minimizer(Cauchy_cumulative_hazard_residual, params, fcn_args=(x, data))
result = minner.minimize()

# calculate final result
forecastdays=forecastDays
final = data + result.residual
x1=np.linspace(1,dataLen+forecastdays,dataLen+forecastdays)
scale=result.params['scale'].value
amp=result.params['amp'].value
loc=result.params['loc'].value
decaybase=result.params['decaybase'].value

y1=amp*Cauchy_cumulative_hazard_fit(x1,loc,scale,decaybase)  # forecast
nn=np.int(np.ceil(loc))
y1[:nn]=final.iloc[:nn] # replace data before

# write error report
report_fit(result)

output = pd.DataFrame({'date' : [],'Forecast':[],'Cases': [],'Fitting':[],'Increase':[]})
output

output['date']=dateObsForecast
output['Forecast']=y1*last
output['Cases'].iloc[:dataLen]=data.values*last
output['Fitting'].iloc[:dataLen]=final.values*last
output['Increase'].iloc[1:]=(y1[1:]-y1[:-1])*last
output=output.set_index('date')

@interact
def plot_ProjectedWorldCOVID19():
    fig=output.iplot(asFigure=True,
             mode='lines+markers',
             size=3,secondary_y = 'Increase',
             secondary_y_title='Increase',
             xTitle='Date',
             yTitle='Cases',
             title='Projected COVID-19 Cases over the World',
             theme='solar',
             filename='COVID19-World',
             sharing='public')
    fig.show()
    url=py.iplot(fig, filename='COVID19-World')
    print(url)

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 216
    # data points      = 107
    # variables        = 4
    chi-square         = 0.01183251
    reduced chi-square = 1.1488e-04
    Akaike info crit   = -966.741443
    Bayesian info crit = -956.050128
[[Variables]]
    amp:        0.11102831 +/- 0.00891390 (8.03%) (init = 1)
    scale:      16.3726793 +/- 1.82742154 (11.16%) (init = 1)
    loc:        61.0295730 +/- 0.65657987 (1.08%) (init = 100)
    decaybase:  0.86638193 +/- 0.01343934 (1.55%) (init = 1)
[[Correlations]] (unreported correlations are < 0.100)
    C(amp, decaybase)   =  0.997
    C(scale, loc)       = -0.944
    C(loc, decaybase)   =  0.922
    C(amp, loc)         =  0.922
    C(scale, decaybase) = -0.861
    C(amp, scale)       = -0.837


interactive(children=(Output(),), _dom_classes=('widget-interact',))